In [1]:
import redcap
from datetime import datetime, timedelta
import pandas as pd
data = {
    'token': 'E5D1F8B694072DCC46C0A8236CCC2670',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
Sen_Data = redcap.Project('https://redcap.mrc.gm:8443/redcap/api/', data['token'])
# IDeA connection

data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

IdeA_Data = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

In [2]:
potential=Sen_Data.export_records(forms=['sensitization'])
potential=pd.DataFrame(potential)
potential=potential[['record_id','part_name_ff615e','mo_name_9185c0','father_699ba1','fat_name_1a8ebe','vill_name_213a8e','mobile1_28ff52','mobile4_4a1123']]
potential['record_id'] = potential['record_id'].str.replace('SEN', '')
potential=pd.DataFrame(potential)

#village_name=potential[['vill_name_213a8e']]
#village_name.to_csv('village_name.csv',index=False)

In [3]:
iron_supplement_v12=IdeA_Data.export_records(forms=['ironsupplement'], event_name=['v12_arm_1'])
iron_supplement_v12=pd.DataFrame(iron_supplement_v12)
iron_supplement_v12_data=iron_supplement_v12[iron_supplement_v12['redcap_event_name']=='v12_arm_1']
iron_supplement_v12_data=iron_supplement_v12_data[['cons_recordid_q1', 'redcap_event_name','redcap_repeat_instance','iron_patcontactdate_q1']]
iron_supplement_v12_data['redcap_repeat_instance'] = pd.to_numeric(iron_supplement_v12_data['redcap_repeat_instance'])
filtered_data = iron_supplement_v12_data[iron_supplement_v12_data['cons_recordid_q1'].str.startswith('IDeA')]
max_instances_by_id = filtered_data.groupby('cons_recordid_q1')['redcap_repeat_instance','redcap_event_name','iron_patcontactdate_q1'].max().reset_index()
max_instances_by_id=max_instances_by_id.rename(columns={'cons_recordid_q1': 'record_id'})

End_line_calls=pd.merge(max_instances_by_id, potential, on='record_id', how='inner')
End_line_callist= pd.read_excel('End_line_Callist.xlsx')
End_line_callist=pd.DataFrame(End_line_callist)

End_line_callist_data= max_instances_by_id[~max_instances_by_id['record_id'].astype(str).isin(End_line_callist['record_id'].astype(str))]
End_line_callist_data=pd.merge(End_line_callist_data,potential,on='record_id', how='inner')
End_line_callist_data.to_csv('End_line_callist_data.csv',index=False)


C:\Users\abliebah\AppData\Local\Temp\ipykernel_700\2452069118.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_instances_by_id = filtered_data.groupby('cons_recordid_q1')['redcap_repeat_instance','redcap_event_name','iron_patcontactdate_q1'].max().reset_index()


In [4]:
iron_supplement=IdeA_Data.export_records(forms=['ironsupplement'])
iron_supplement=pd.DataFrame(iron_supplement)
iron_supplement=iron_supplement[['cons_recordid_q1', 'redcap_event_name', 'redcap_repeat_instrument',
       'redcap_repeat_instance', 'iron_patcontactdate_q1']]
iron_supplement=iron_supplement.rename(columns={'cons_recordid_q1': 'record_id'})
iron_supplement=iron_supplement[
    (iron_supplement['redcap_event_name']=='v1_arm_1' ) &
    (iron_supplement['redcap_repeat_instance']==1)
]


In [5]:
iron_supplement['start_date'] = pd.to_datetime(iron_supplement['iron_patcontactdate_q1'])
iron_supplement['due_date'] = iron_supplement['start_date'] + pd.to_timedelta(83, unit='D')
iron_supplement = iron_supplement[iron_supplement['record_id'].str.startswith('IDeA')]
End_line_callist_data=pd.merge(potential,iron_supplement ,on='record_id', how='inner')
End_line_callist_data.to_csv('End_line_callist_data.csv',index=False)


In [6]:

# Filter sysmext_data to include only records where 'sysmext_data' is completed ('sysmexresult_complete' equals '2')
sysmext_data = IdeA_Data.export_records(forms=['sysmexresult'])
sysmext_data = pd.DataFrame(sysmext_data)
sysmext_data = sysmext_data.rename(columns={'cons_recordid_q1': 'record_id'})
sysmext_data = sysmext_data[
    (sysmext_data['sysmexresult_complete'] == '2') &
    (sysmext_data['redcap_event_name'] == 'v12_arm_1')
]





In [7]:
result_record_ids = record_ids_more_than_84_days[~record_ids_more_than_84_days.isin(sysmext_data['record_id'])]

NameError: name 'record_ids_more_than_84_days' is not defined